In [5]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

In [6]:
from string import Template

def load_template(filename: str) -> Template:
    file = open(filename, "r")
    content = file.read()
    file.close()
    template = Template(content)
    return template

In [13]:
def get_questions(profile_name:str) -> str:
    questions = ""
    if profile_name == "profile1":
        questions = """
            option1: question about the status of a parcel
            option7: question about the location of a parcel
        """
    elif profile_name == "profile2":
        questions = """
            option1: question about the status of a parcel
            option2: question about the status of a refund
            option3: question about the status of a return
            option4: question about the status of a complaint
            option5: question about the status of a product
            option6: question about the taxes to pay
            option7: question about the location of a parcel
        """
    return questions

def get_prompt(profile_name: str) -> str:
    template_file_name = "./prompts/main_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(questions=get_questions(profile_name))
    return prompt

def call_openAI(profile_name, text):
    system_message = get_prompt(profile_name)  

    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
        messages = [
            {"role":"system","content":system_message},
            {"role":"user","content":text}
            ],
        temperature=0,
    )

    return response.choices[0].message.content


In [14]:
# profile1 accepts only questions about the status of a parcel and its location
query =  "¿Dónde está mi paquete?"
answer = call_openAI("profile1", query)
display(HTML(answer))

In [15]:
# profile1 accepts only questions about the status of a parcel and its location - so option 99
query =  "I wamt to know the status of my refund"
answer = call_openAI("profile1",query)
display(HTML(answer))

In [16]:
#profile2
query =  "quais são os impostos a pagar pela minha encomenda"
answer = call_openAI("profile2",query)
display(HTML(answer))